# Second: Write queries that directly answer predetermined questions from a business stakeholder
Before writing queries, it is essential to understand the JSON data and transform it into a structured relational model which is done as part of Task 1. The script below is used to load JSON files, flatten the data, and convert them into CSV files. These structured CSV files can then be uploaded into the MySQL database for writing the SQL queries required in Task 2.

In [2]:
import json
import csv
from datetime import datetime

# Function to load MongoDB JSON data
def load_mongo_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = []
        # Read each line and load as separate JSON objects
        for line in file:
            data.append(json.loads(line))
        return data

# Function to write data to CSV
def write_to_csv(file_name, data, fieldnames):
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()  # Write the header
        writer.writerows(data)  # Write the data rows

# Function to convert MongoDB data to a structured format compatible with SQL
def segregate_data(users_file, brands_file, receipts_file):
    # Load original MongoDB JSON data
    users_data = load_mongo_json(users_file)
    brands_data = load_mongo_json(brands_file)
    receipts_data = load_mongo_json(receipts_file)

    # Extract Users data (Flatten MongoDB document)
    usersnew = []
    for user in users_data:
        usersnew.append({
           "user_id": user["_id"]["$oid"],  # MongoDB ObjectID ($oid) converted to string
            "active": user.get("active", None),
            "created_Date": convert_mongo_date(user.get("createdDate", None)),
            "last_Login": convert_mongo_date(user.get("lastLogin", None)),
            "role": user.get("role", None),
            "sign_Up_Source": user.get("signUpSource", None),
            "state": user.get("state", None)
        })
    
    # Define the fieldnames for Users CSV
    users_fieldnames = ["user_id", "active", "created_Date", "last_Login", "role", "sign_Up_Source", "state"]
    write_to_csv("users_normalized.csv", usersnew, users_fieldnames)

    # Extract Brands and CPGs data
    brandsnew = []
    cpgs = {}
    for brand in brands_data:
        brandsnew.append({
            "brand_id": brand["_id"]["$oid"],  # MongoDB ObjectID ($oid) converted to string
            "bar_Code": brand.get("barcode", None),
            "brand_Code": brand.get("brandCode", None),
            "category": brand.get("category", None),
            "category_Code": brand.get("categoryCode", None),
            "name": brand.get("name", None),
            "top_Brand": brand.get("topBrand", None),
            "cpg_id": brand.get("cpg", {}).get("$id", {}).get("$oid", None)  # Handle CPG reference
        })
        # Add unique CPGs to the CPGs table
        cpgs[brand.get("cpg", {}).get("$id", {}).get("$oid", None)] = {
            "cpg_id": brand.get("cpg", {}).get("$id", {}).get("$oid", None), 
            "name": brand.get("cpg", {}).get("$ref", None)
        }

    # Define the fieldnames for Brands CSV
    brands_fieldnames = ["brand_id", "bar_Code", "brand_Code", "category", "category_Code", "name", "top_Brand", "cpg_id"]
    write_to_csv("brands_normalized.csv", brandsnew, brands_fieldnames)
    
    # Write CPGs data to CSV
    cpgs_list = list(cpgs.values())
    cpgs_fieldnames = ["cpg_id", "name"]
    write_to_csv("cpgs_normalized.csv", cpgs_list, cpgs_fieldnames)
    
    # Extract Receipts and ReceiptItems data
    receiptsnew  = []
    receipt_items = []
    for receipt in receipts_data:
        receiptsnew.append({
            "_id": receipt["_id"]["$oid"],  # MongoDB ObjectID ($oid) converted to string
            "bonus_Points_Earned": receipt.get("bonusPointsEarned", None),
            "bonusPoints_Earned_Reason": receipt.get("bonusPointsEarnedReason", None),
            "create_Date": convert_mongo_date(receipt.get("createDate", None)),
            "date_Scanned": convert_mongo_date(receipt.get("dateScanned", None)),
            "finished_Date": convert_mongo_date(receipt.get("finishedDate", None)),
            "modify_Date": convert_mongo_date(receipt.get("modifyDate", None)),
            "points_Awarded_Date": convert_mongo_date(receipt.get("pointsAwardedDate", None)),
            "points_Earned": receipt.get("pointsEarned", None),
            "purchase_Date": convert_mongo_date(receipt.get("purchaseDate", None)),
            "purchased_Item_Count": receipt.get("purchasedItemCount", None),
            "rewards_Receipt_Status": receipt.get("rewardsReceiptStatus", None),
            "total_Spent": receipt.get("totalSpent", None),
            "user_Id": receipt.get("userId", None)
        })
        
        # Safely access 'rewardsReceiptItemList'
        rewardsReceiptItemList = receipt.get("rewardsReceiptItemList", [])  # Default to empty list if missing
        for item in rewardsReceiptItemList:
            receipt_items.append({
                "item_id": item.get("partnerItemId", None),
                "receipt_id": receipt["_id"]["$oid"],  # MongoDB ObjectID converted to string
                "barcode": item.get("barcode", None),
                "description": item.get("description", None),
                "final_Price": item.get("finalPrice", None),
                "item_Price": item.get("itemPrice", None),
                "quantity_Purchased": item.get("quantityPurchased", None),
                "needs_Fetch_Review": item.get("needsFetchReview", None),
                "partner_Item_id": item.get("partnerItemId", None),
                "prevent_Target_Gap_Points": item.get("preventTargetGapPoints", None),
                "user_Flagged_Barcode": item.get("userFlaggedBarcode", None),
                "user_Flagged_Description": item.get("userFlaggedDescription", None),
                "user_Flagged_New_Item": item.get("userFlaggedNewItem", None),
                "user_Flagged_Price": item.get("userFlaggedPrice", None),
                "user_Flagged_Quantity": item.get("userFlaggedQuantity", None),
                "points_Not_Awarded_Reason": item.get("pointsNotAwardedReason", None),
                "points_Payer_id": item.get("pointsPayerId", None),
                "rewards_Group": item.get("rewardsGroup", None),
                "rewards_Product_Partner_id": item.get("rewardsProductPartnerId", None)
            })

    # Define the fieldnames for Receipts CSV
    receipts_fieldnames = ["_id", "bonus_Points_Earned", "bonusPoints_Earned_Reason", "create_Date", "date_Scanned", 
                           "finished_Date", "modify_Date", "points_Awarded_Date", "points_Earned", "purchase_Date", 
                           "purchased_Item_Count", "rewards_Receipt_Status", "total_Spent", "user_Id"]
    write_to_csv("receipts_normalized.csv", receiptsnew, receipts_fieldnames)
    
    # Define the fieldnames for ReceiptItems CSV
    receipt_items_fieldnames = ["item_id", "receipt_id", "barcode", "description", "final_Price", "item_Price", 
                                "quantity_Purchased", "needs_Fetch_Review", "partner_Item_id", "prevent_Target_Gap_Points", 
                                "user_Flagged_Barcode", "user_Flagged_Description", "user_Flagged_New_Item", 
                                "user_Flagged_Price", "user_Flagged_Quantity", "points_Not_Awarded_Reason", "points_Payer_id", 
                                "rewards_Group", "rewards_Product_Partner_id"]
    write_to_csv("receipt_items_normalized.csv", receipt_items, receipt_items_fieldnames)

    print("Data segregation is completed and Files are saved as per below:")
    print("users_normalized.csv")
    print("brands_normalized.csv")
    print("cpgs_normalized.csv")
    print("receipts_normalized.csv")
    print("receipt_items_normalized.csv")

# Function to convert MongoDB date format (timestamps) to SQL-friendly format
def convert_mongo_date(mongo_date):
    if mongo_date:
        if isinstance(mongo_date, dict) and "$date" in mongo_date:
            # Extract the timestamp from the $date field and convert it to datetime
            return datetime.utcfromtimestamp(mongo_date["$date"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
    return None

# Paths to your original MongoDB JSON files
users_file = 'users.json'
brands_file = 'brands.json'
receipts_file = 'receipts.json'

# Segregate the data
segregate_data(users_file, brands_file, receipts_file)


Data segregation is completed and Files are saved as per below:
users_normalized.csv
brands_normalized.csv
cpgs_normalized.csv
receipts_normalized.csv
receipt_items_normalized.csv
